In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [31]:
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI

chat = ChatOpenAI()
llm = OpenAI()


In [12]:
from langchain.output_parsers import DatetimeOutputParser

output_parser = DatetimeOutputParser()

In [5]:
output_parser.get_format_instructions()

'Write a datetime string that matches the \n            following pattern: "%Y-%m-%dT%H:%M:%S.%fZ". Examples: 1060-01-16T01:55:06.262421Z, 1607-10-31T08:41:21.962334Z, 1036-03-16T08:21:51.478016Z'

In [6]:
from langchain.prompts.chat import HumanMessagePromptTemplate, ChatPromptTemplate

In [7]:
template_text = "{request} \n {format_instructions}"
human_prompt = HumanMessagePromptTemplate.from_template(template_text)

In [8]:
chat_prompt = ChatPromptTemplate.from_messages([human_prompt])

In [16]:
model_request = chat_prompt.format_prompt(request="what date was the 13th ammendment ratified in the US?", format_instructions=output_parser.get_format_instructions()).to_messages()

In [17]:
result = chat(model_request, temperature=0)


In [18]:
result.content

'The 13th Amendment was ratified in the US on December 6, 1865. \n\nThe datetime string that matches the given pattern "%Y-%m-%dT%H:%M:%S.%fZ" for this date would be "1865-12-06T00:00:00.000000Z".'

This response is an error, it was supposed to give a datetime


We could fix it by providing a system prompt, or using OutputFixingParser

In [26]:
from langchain.output_parsers import OutputFixingParser

In [20]:
misformatted = result.content

In [32]:
new_parser = OutputFixingParser.from_llm(parser=output_parser, llm=llm)

In [28]:
misformatted

'The 13th Amendment was ratified in the US on December 6, 1865. \n\nThe datetime string that matches the given pattern "%Y-%m-%dT%H:%M:%S.%fZ" for this date would be "1865-12-06T00:00:00.000000Z".'

In [34]:
new_parser.parse(misformatted)

OutputParserException: Could not parse datetime string: 

Write a datetime string that matches the following pattern: "%Y-%m-%dT%H:%M:%S.%fZ". Examples: 1990-01-29T15:36:53.434334Z, 0331-10-21T19:32:12.244030Z, 1314-06-26T03:01:59.608139Z

So that didnt work. I'm going to add a system message

In [35]:
from langchain.prompts import SystemMessagePromptTemplate

system_prompt = SystemMessagePromptTemplate.from_template("You only reply to questions with a datetime.")


In [36]:
chat_prompt = ChatPromptTemplate.from_messages([system_prompt, human_prompt])

In [38]:
model_request = chat_prompt.format_prompt(request="what date was the 13th ammendment ratified in the US?", format_instructions=output_parser.get_format_instructions()).to_messages()
result = chat(model_request, temperature=0)


In [40]:
output_parser.parse(result.content)

datetime.datetime(1865, 12, 6, 0, 0)